# Extraer planos nodales de sismos usando el "product type" de tensores de momento en la págian del NEIC
---

## Dependencias del código

In [6]:
from datetime import datetime
from libcomcat.search import search, get_event_by_id
from libcomcat.dataframes import get_summary_data_frame
import pandas as pd

## Parámetros de búsqueda (fecha, latitud, longitud y magnitud)

In [7]:
starttime = datetime(1970, 1, 1, 0, 0, 0) #yyyy, mm, dd,hh, mm, ss
endtime = datetime(2024, 12, 31, 0, 0, 0)

min_latitude = 4.675
max_latitude = 12.147
min_longitude = -88.982
max_longitude = -81.101
min_mag = 5.5
max_mag = 8

events = search(starttime=starttime, endtime=endtime,
                maxlatitude=max_latitude, minlatitude=min_latitude,
                maxlongitude=max_longitude, minlongitude=min_longitude,
                minmagnitude=min_mag, maxmagnitude=max_mag,
                producttype='moment-tensor')

event_data = []

for event in events:
    try:
        event_details = get_event_by_id(event.id)

        moment_tensor_products = event_details.getProducts('moment-tensor')
        
        if moment_tensor_products:
            moment_tensor_product = moment_tensor_products[0]

            strike1 = moment_tensor_product['nodal-plane-1-strike']
            dip1 = moment_tensor_product['nodal-plane-1-dip']
            rake1 = moment_tensor_product['nodal-plane-1-rake']

            strike2 = moment_tensor_product['nodal-plane-2-strike']
            dip2 = moment_tensor_product['nodal-plane-2-dip']
            rake2 = moment_tensor_product['nodal-plane-2-rake']
            
            event_info = {
                'id': event.id,
                'time': event.time,
                'latitude': event.latitude,
                'longitude': event.longitude,
                'depth': event.depth,
                'magnitude': event.magnitude,
                'strike_1': strike1,
                'dip_1': dip1,
                'rake_1': rake1,
                'strike_2': strike2,
                'dip_2': dip2,
                'rake_2': rake2
                }

            event_data.append(event_info)
        else:
            print(f"Evento {event.id} no tiene un producto de tipo moment-tensor")

    except Exception as e:
        print(f"Error al procesar evento {event.id}: {str(e)}")

## Ver la lista de eventos procesados

In [ ]:
print("Lista de eventos:")
for event in event_data:
    print(event)

## Guardar la lista de eventos en un .csv

In [8]:
events_df = pd.DataFrame(event_data)

# Acá se puede cambiar el nombre del archivo, este se guarda en la misma carpeta en la que se encuentra este notebook

csv_file = 'sismos_tensores_de_momento.csv'
events_df.to_csv(csv_file, index=False)

print(f"Datos guardados exitosamente en '{csv_file}'")

Datos guardados exitosamente en 'sismos_tensores_de_momento.csv'
